In [ ]:
!mkdir -p /tmp/pip/cache/
!mkdir -p /root/.cache/torch/hub/checkpoints
!cp ../input/pawpularity-dataset-first-model/efficientnet_pytorch-0.7.1.xyz /tmp/pip/cache/efficientnet_pytorch-0.7.1.tar.gz
!cp ../input/pawpularity-dataset-first-model/efficientnet-b0-355c32eb.pth /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet_pytorch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import albumentations
import torch.nn as nn
import efficientnet_pytorch
import torch.nn.functional as F
import gc
import warnings

from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm

warnings.filterwarnings("ignore")

In [ ]:
class config:
    TESTING_FILE = "../input/petfinder-pawpularity-score/test.csv"
    SUBMISSION_FILE = '../input/petfinder-pawpularity-score/sample_submission.csv'
    TESTING_IMAGE_PATH = "../input/petfinder-pawpularity-score/test/"
    DEVICE = torch.device("cuda")
    TEST_BATCH_SIZE = 2

In [ ]:
class petFinderDataset:
    def __init__(self, constant_func, dataframe):
        self.constant_func = constant_func
        self.dataframe = dataframe
        self.aug = albumentations.Compose([
            albumentations.RandomResizedCrop(224, 224),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.HueSaturationValue(
                hue_shift_limit=0.2, 
                sat_shift_limit=0.2, 
                val_shift_limit=0.2, 
                p=0.5
            ),
            albumentations.RandomBrightnessContrast(
                brightness_limit=(-0.1,0.1), 
                contrast_limit=(-0.1, 0.1), 
                p=0.5
            ),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
            albumentations.CoarseDropout(p=0.5),
            albumentations.Cutout(p=0.5)], p=1.)
    
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, item):
        df = self.dataframe.iloc[item, :]

        # converting jpg format of images to numpy array
        img = np.array(Image.open(self.constant_func.TESTING_IMAGE_PATH + df["Id"] + '.jpg')) 
        img = self.aug(image = img)['image']
        img = np.transpose(img , (2,0,1)).astype(np.float32) # 2,0,1 because pytorch excepts image channel first then dimension of image
    
        return {
            'image': torch.tensor(img, dtype = torch.float),
            'tabular_data' : torch.tensor(df[1:], dtype = torch.float)
        }

new_train = pd.read_csv(config.TESTING_FILE)
dataset = petFinderDataset(config(), new_train)
print(dataset[7]['tabular_data'])
img = dataset[7]['image']
plt.imshow(np.transpose(img.numpy(), (1,2,0)))

In [ ]:
test_dataloader = DataLoader(dataset,
                            num_workers=4,
                            batch_size=config.TEST_BATCH_SIZE,
                            shuffle=False)

In [ ]:
class EfficientNet_b0(nn.Module):
    def __init__(self):
        super(EfficientNet_b0, self).__init__()
        self.model = efficientnet_pytorch.EfficientNet.from_pretrained('efficientnet-b0')
        self.dropout = nn.Dropout(0.1)
        self.final_layer = nn.Linear(1280 , 1)
        
        self.tabular_dense_layer_1 = nn.Linear(12, 8)
        self.tabular_dense_layer_2 = nn.Linear(8, 4)
        self.tabular_dense_layer_3 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        
        self.outputs = nn.Linear(2 , 1)
        
    def forward(self, inputs ,tabular_data_inputs):
        batch_size, _, _, _ = inputs.shape
        
        x = self.model.extract_features(inputs)

        # Pooling and final linear layer
        x = self.model._avg_pooling(x)
        
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        x = self.final_layer(self.dropout(x))
        
        tab = self.tabular_dense_layer_1(tabular_data_inputs)
        tab = self.relu(tab)
        tab = self.tabular_dense_layer_2(tab)
        tab = self.relu(tab)
        tab = self.tabular_dense_layer_3(tab)
        tab = self.relu(tab)
        
        op = torch.cat((x, tab), dim=1)
        op = self.relu(op)

        return self.outputs(op)
    
model = EfficientNet_b0()
model = model.to(config.DEVICE)

In [ ]:
checkpoint_path = "../input/pawpularity-dataset-first-model/first_basic_model.bin"

model = torch.load(checkpoint_path)

In [ ]:
# run inference 5 times for test time augmentation implementation
model.eval()

final_preds = None
for i in range(5):
    for batch_index,dataset in enumerate(test_dataloader):
        tabular_data = dataset['tabular_data']
        image = dataset['image']
        
        tabular_data = tabular_data.to(config.DEVICE, dtype=torch.float)
        image = image.to(config.DEVICE, dtype=torch.float)

        with torch.no_grad():
            preds = model(image, tabular_data)
        if batch_index == 0:
            temp_preds = preds.detach().cpu().numpy()
        else:
            temp_preds = np.vstack((temp_preds , preds.detach().cpu().numpy()))
    if i == 0:
        final_preds = temp_preds
    else:
        final_preds += temp_preds
        
final_preds /= 5

In [ ]:
sample_submission = pd.read_csv(config.SUBMISSION_FILE)
sample_submission.Pawpularity = final_preds 
sample_submission

In [ ]:
sample_submission.to_csv("submission.csv", index=False)